In [1]:
#%%file test.py
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression

X, y = load_diabetes(True)

lr = LinearRegression()
lr.fit(X, y)
import algolink
model = algolink.create_model(lr, X, model_name='diabetes_model_11')
model

/Users/leepand/miniconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass return_X_y=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Model(id=None,name=diabetes_model_11)

In [2]:
from algolink.ext.flask.client import HTTPClient


In [4]:
client = HTTPClient('localhost', 9000)


In [5]:
client.methods['predict']


Method(name='predict', args=[Argument(name='vector', type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None, 10),dtype=float64]'>)], out_type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None,),dtype=float64]'>)

In [6]:
import numpy as np

client.predict(np.array([[0., 1., 2., 3., 4., 5., 6., .7, .8, .9]]))

array([2532.25644396])

In [8]:
from pprint import pprint

import numpy as np
import pandas as pd
from pyjackson import serialize
from sklearn.metrics import roc_auc_score

import algolink


def get_data():
    data = pd.DataFrame([[1, 0], [0, 1]], columns=['a', 'b'])
    target = np.array([1, 0])
    return data, target


def constant(data):
    return np.array([0 for _ in range(len(data))])


def truth(data: pd.DataFrame):
    return np.array([r[0] for _, r in data.iterrows()])


def my_custom_metric(y_true, y_score):
    return y_score.sum() / y_true.sum() * 100.


def main():
    ebnt = algolink.AlgoLink.local(clear=True)

    data, target = get_data()
    # we want easy way to transform anything to datasets, so its either this or ebonite.create_dataset (same for metrics)
    # for now there is no difference, however if we want manage datasets with meta and art repos, we use client
    # or create with ebonite.create_... and then push with ebnt.push_... like for models
    # dataset = ebnt.create_dataset(data, target)

    # here we postpone setting task input and output types for easy task creation
    task = ebnt.get_or_create_task('my_project', 'regression_is_my_profession')
    print(roc_auc_score,"roc_auc_score")
    task.add_metric('auc', roc_auc_score)
    task.add_metric('custom', my_custom_metric)
    task.add_evaluation('train', data, target, ['auc', 'custom'])

    pprint(task.evaluation_sets)
    pprint(task.datasets)
    pprint(task.metrics)

    # omit providing dataset as we already have it in task
    mc = task.create_and_push_model(constant, data, model_name='constant')
    mt = task.create_and_push_model(truth, data, model_name='truth')

    pprint(mc.wrapper.methods)
    pprint(mt.wrapper.methods)

    # maybe save result to models? also need different ways to evaluate "not all"
    result = task.evaluate_all()

    print(result)
    ebnt._bind(task)
    task.save()
    pprint(serialize(task))


if __name__ == '__main__':
    main()

<function roc_auc_score at 0x7fd75ae49dd0> roc_auc_score
{'train': <algolink.core.objects.core.EvaluationSet object at 0x7fd75d585550>}
{'train_input': <algolink.core.objects.dataset_source.InMemoryDatasetSource object at 0x7fd75d585950>,
 'train_output': <algolink.core.objects.dataset_source.InMemoryDatasetSource object at 0x7fd75d588cd0>}
{'auc': <algolink.core.objects.metric.LibFunctionMetric object at 0x7fd75d5852d0>,
 'custom': <algolink.core.objects.metric.CallableMetric object at 0x7fd75d585d50>}
{'predict': ('__call__',
             <class 'pyjackson.generics.DataFrameType[columns=['a', 'b'],dtypes=['int64', 'int64'],index_cols=[]]'>,
             <class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(2,),dtype=int64]'>)}
{'predict': ('__call__',
             <class 'pyjackson.generics.DataFrameType[columns=['a', 'b'],dtypes=['int64', 'int64'],index_cols=[]]'>,
             <class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(2,),dtype=int64]'>)}
{'constant': Evaluation

In [9]:
from algolink.ext.sqlalchemy.repository import SQLAlchemyMetaRepository


In [12]:
test = SQLAlchemyMetaRepository('sqlite:///sql.db')

In [13]:
!pwd

/Users/leepand/Downloads/MLOps/AlgoLink/src


In [18]:
import os

import numpy as np

import algolink
from algolink import AlgoLink


def add_one(data):
    return data + 1


def main():
    #  create remote ebonite client. This client stores metadata in postgres and artifacts in s3
    os.environ['S3_ACCESS_KEY'] = 'eboniteAccessKey'
    os.environ['S3_SECRET_KEY'] = 'eboniteSecretKey'
    alink = AlgoLink.custom_client('sqlalchemy', 'local',
                                 meta_kwargs={'db_uri': 'sqlite:///sql.db'},
                                 artifact_kwargs={'path': './'})
    # save client configuration for later use
    alink.save_client_config('client_config.json')
    #  obtain Task
    task = alink.get_or_create_task('my_project', 'my_task')

    #  remove model if it exists (for demo purposes)
    if task.models.contains('add_one_model'):
        model = task.models('add_one_model')
        task.delete_model(model)

    #  create model from function add_one and numpy array as data sample
    model = algolink.create_model(add_one, np.array([0]), 'add_one_model')

    #  persist model
    task.push_model(model)


if __name__ == '__main__':
    main()

In [19]:
import os

from algolink import AlgoLink
from algolink.runtime import run_model_server


def main():
    #  create remote ebonite client from saved configuration
    os.environ['S3_ACCESS_KEY'] = 'eboniteAccessKey'
    os.environ['S3_SECRET_KEY'] = 'eboniteSecretKey'
    ebnt = AlgoLink.from_config_file('client_config.json')
    model = ebnt.get_model('add_one_model', 'my_task', 'my_project')

    # run flask service with this model
    run_model_server(model)
    # now you can use client.py to call this service or go to http://localhost:9000/apidocs to view swagger ui


if __name__ == '__main__':
    main()


2021-09-24 17:54:29,348 [INFO] algolink_runtime: Starting AlgoLink runtime with loader DummyLoader and server FlaskServer ...
2021-09-24 17:54:29,356 [INFO] algolink_runtime: Running server <algolink.ext.flask.server.FlaskServer object at 0x7fd75e089750>


[2021-09-24 17:54:29 +0800] [3346] [INFO] Starting gunicorn 20.0.4
[2021-09-24 17:54:29 +0800] [3346] [INFO] Listening at: http://0.0.0.0:9000 (3346)
[2021-09-24 17:54:29 +0800] [3346] [INFO] Using worker: gevent
[2021-09-24 17:54:29 +0800] [16463] [INFO] Booting worker with pid: 16463
[2021-09-24 17:54:29 +0800] [16464] [INFO] Booting worker with pid: 16464
/Users/leepand/miniconda3/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:53: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.contrib.pyopenssl (/Users/leepand/miniconda3/lib/python3.7/site-packages/urllib3/contrib/pyopenssl.py)', 'botocore.httpsession (/Users/leepand/miniconda3/lib/python3.7/site-packages/botocore/httpsession.py)']. 
  monk

SystemExit: 1

/Users/leepand/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
[2021-09-24 19:31:09 +0800] [18651] [INFO] Booting worker with pid: 18651
/Users/leepand/miniconda3/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:53: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.contrib.pyopenssl (/Users/leepand/miniconda3/lib/python3.7/site-packages/urllib3/contrib/pyopenssl.py)', 'botocore.httpsession (/Users/leepand/miniconda3/lib/python3.7/site-packages/botocore/httpsession.py)']. 
  monkey.patch_all()
[2021-09-24 19:38:54 +0800] [3346] [INFO] Handling signal: int

SystemExit: 0

...

SystemExit: 0

/Users/leepand/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
[2021-09-24 19:38:55 +0800] [3346] [INFO] Shutting down: Master
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/leepand/miniconda3/lib/python3.7/site-packages/gunicorn/arbiter.py", line 224, in run
    handler()
  File "/Users/leepand/miniconda3/lib/python3.7/site-packages/gunicorn/arbiter.py", line 262, in handle_int
    raise StopIteration
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/leepand/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-4217764d64b2>", line 20, in <module>
    main()
  File "<ipython-input-19-4217764d64b2>", line 15, in main
    run_model_server(model)
  File "/Users/leepand/Downloads/MLOps/AlgoLink/src/algolink/runtime/helpers.py", line 33, in run_model_server
    start_runtime(DummyLoader(), server)
  File "/Users/leepand/Downloads/MLOps/AlgoLink/src/algolink/runtime/command_line.py", line 29, in start_runtime

SystemExit: 0

/Users/leepand/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
!pwd

ExceptionPexpect: isalive() encountered condition where "terminated" is 0, but there was no child process. Did someone else call waitpid() on our process?